## 지하철 역별 시간별 하차 인원수 예측
#### 필요 데이터(중요도 순)
>##### 1. 과거 시간대별 역별 일별 승하차 데이터
>##### 2. 날짜 및 시간 정보(요일, 공휴일, 계절 등), 날씨
>##### 3. 특별 행사 여부, 인구 통계 및 인프라(업무지구, 학교 등)

In [1]:
#서울 지하철 호선별 역별 승하차 정보
#https://data.seoul.go.kr/dataList/OA-12914/S/1/datasetView.do
#서울 지하철 현재 위치 정보
#https://data.seoul.go.kr/dataList/OA-12601/A/1/datasetView.do

### IMPORT

In [2]:
%matplotlib inline
import apikey
import requests
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
from datetime import datetime
import xmltodict
import matplotlib as plt
import seaborn as sns

### 데이터 ETL

#### 1. 과거 지하철 데이터 관련

In [3]:
def timeCos(x): # 시간대 코사인 변환(24시와 5시의 시간적 차이를 줄이기 위해서)
    return np.cos(int(x) * np.pi / 12.0)

def checkWeekend(x): # 주말여부 확인 밑 요일 리턴
    year, mon, day = x.split('-')
    date = datetime(int(year),int(mon),int(day))
    day_week = date.weekday()
    if day_week >= 5:
        return 1, day_week
    else:
        return 0, day_week

In [4]:
# 미세먼지 농도 연도별 병합 함수
'''
def dust(year,month):
    df = pd.read_excel('subway/미세먼지/' + str(year) + '/' + str(month) + '월.xlsx',skiprows=4)
    df = df.drop(['Unnamed: 1'],axis=1)
    df = df.rename(columns=lambda x:(str(year) + '-' + str(month).zfill(2) + '-' + x[:-1].zfill(2)))
    df = df.rename(columns={df.columns[0]:'위치'})
    df = df.T
    df = df.rename(columns=df.iloc[0])
    df = df.drop(df.index[0])
    return df
'''
#df_dust = dust(2023,1)

"\ndef dust(year,month):\n    df = pd.read_excel('subway/미세먼지/' + str(year) + '/' + str(month) + '월.xlsx',skiprows=4)\n    df = df.drop(['Unnamed: 1'],axis=1)\n    df = df.rename(columns=lambda x:(str(year) + '-' + str(month).zfill(2) + '-' + x[:-1].zfill(2)))\n    df = df.rename(columns={df.columns[0]:'위치'})\n    df = df.T\n    df = df.rename(columns=df.iloc[0])\n    df = df.drop(df.index[0])\n    return df\n"

In [5]:
# 미세먼지 불러오기 및 결측값 처리
def dustDf(year):
    df_dust = pd.read_csv('subway/미세먼지/' + str(year) + '/미세먼지_' + str(year) + '.csv',encoding='utf-8')
    df_dust.set_index(df_dust.columns[0],inplace=True)
    df_dust.index.name = None
    # 결측값 처리
    df_dust = df_dust.ffill()
    return df_dust

In [6]:
# 역별 소속구
def getLoc(x):
    temp = x.split()
    return temp[1]

df_station_loc = pd.read_csv('subway/역주소.csv',encoding='cp949')
df_station_loc.drop(['연번','역전화번호','지번주소'],axis=1,inplace=True)
df_station_loc['도로명주소'] = df_station_loc['도로명주소'].apply(getLoc)
df_station_loc = df_station_loc.rename({'도로명주소':'지역'},axis=1)
df_station_loc.replace({'역명':'서울'},'서울역',inplace=True)
df_station_loc.replace({'역명':'교대(법원, 검찰청)'},'교대(법원.검찰청)',inplace=True)
df_station_loc.replace({'역명':'남한산성입구(성남법원?검찰청)'},'남한산성입구(성남법원.검찰청)',inplace=True)
df_station_loc.replace({'역명':'신촌(지하)'},'신촌',inplace=True)
df_station_loc.replace({'역명':'하남시청(덕풍?신장)'},'하남시청(덕풍·신장)',inplace=True)
df_station_loc.drop(['역번호','호선'],axis=1,inplace=True)
df_station_loc.drop_duplicates(subset='역명',inplace=True)
df_station_loc.head(5)

,역명,지역
0,서울역,중구
1,시청,중구
2,종각,종로구
3,종로3가,종로구
4,종로5가,종로구


In [7]:
# 구별 인구수
df_population = pd.read_csv('subway/행정구역_시군구_별_주민등록세대수.csv',encoding='utf-8')
df_population.drop(['2023.11','2024.01','2024.02'],axis=1,inplace=True)
df_population.drop(df_population.index[27:91],axis=0,inplace=True)
df_population.rename(columns={'행정구역(시군구)별':'지역','2023.12':'인구수'},inplace=True)
df_population.head(5)

,지역,인구수
0,전국,23914851
1,서울특별시,4469417
2,종로구,72067
3,중구,64714
4,용산구,107825


In [8]:
df_station_population = pd.merge(df_station_loc,df_population, on='지역', how='left')
df_station_population.drop(['지역'],axis=1,inplace=True)
df_station_population.head(5)

,역명,인구수
0,서울역,64714
1,시청,64714
2,종각,72067
3,종로3가,72067
4,종로5가,72067


In [9]:
def subwayDf(year):
    locate = 'subway/역별 일별 시간대별 승하차인원/역별 일별 시간대별 승하차인원_' + str(year) + '.csv'
    df = pd.read_csv(locate,encoding='EUC-KR')
    if year == 2021:
        df = df.rename(columns={'구분':'승하차구분','날짜':'수송일자'})
    df_out = df[df['승하차구분']=='하차']
    if year == 2021:
        df_out = df_out.drop(['연번','승하차구분','06시 이전','23시 이후','합 계'],axis=1)
    else:
        df_out = df_out.drop(['연번','승하차구분','06시이전','24시이후'],axis=1)
    if year == 2022:
        df_out.drop(df_out[df_out['역번호']==' '].index, inplace=True)
        df_out['역번호'] = df_out['역번호'].apply(int)
    # 날짜 -> 요일과 주말여부 칼럼 추가
    isWeekend = []
    whatWeek = []
    for s in df_out['수송일자']:
        weekend, day_week = checkWeekend(s)
        isWeekend.append(weekend)
        whatWeek.append(day_week)
    df_out['주말'] = isWeekend # 토or일 : 1
    df_out['요일'] = whatWeek
    
    # 해당 지하철역이 몇개의 환승노선으로 이루어져있는지 수
    df_trans = pd.read_csv('subway/노선별지하철역정보.csv', encoding='cp949')
    df_trans =  df_trans[['전철역명']]
    df_trans = df_trans.rename(columns={'전철역명':'역명'})
    df_trans_counts = df_trans['역명'].value_counts()
    df_trans['통과호선수'] = df_trans['역명'].map(df_trans_counts)
    df_trans.drop_duplicates(subset='역명',inplace=True)
    df_out = df_out.merge(df_trans, on='역명', how='left').fillna({'통과호선수':1})
    df_out['통과호선수'] = df_out['통과호선수'].apply(int)
    
    # 시간대 이름 설정
    df_out = df_out.rename(columns={'수송일자':'일시'})
    start_index = 4
    end_index = 21
    new_columns = [col if idx < start_index or idx > end_index else col[:2] for idx, col in enumerate(df_out.columns)]
    df_out.columns = new_columns
    
    # 역이 소속된 구의 인구수 추가
    #df_out = df_out.merge(df_station_population, on='역명', how='left')
    
    # 역별 시간대별 미세먼지 추가(PM-10)
    df_dust = dustDf(year)
    df_out = df_out.merge(df_station_loc, on='역명', how='left')
    dust_arr = []
    for i,date in enumerate(df_out['일시']):
        try:
            dust_arr.append(df_dust.loc[date][df_out['지역'].iloc[i]])
        except:
            dust_arr.append(df_dust.loc[date]['서울시 평균'])
    df_out['미세먼지(pm10)'] = dust_arr
    df_out.drop(['지역'], axis=1, inplace=True)
    
    # 시간대 한 컬럼으로 통합 및 하차인원 분리
    df_out_melted = pd.melt(df_out,id_vars=['일시','호선','역번호','역명','주말','요일','통과호선수','미세먼지(pm10)'],var_name='시간',value_name='하차인원')
    
    # 시간대 코사인 함수 적용
    #df_out_melted['시간'] = df_out_melted['시간'].apply(timeCos)
    
    # 호선 숫자로 변경
    if year == 2023:
        df_out_melted['호선'] = df_out_melted['호선'].apply(lambda x:int(x[0]))
    return df_out_melted

In [10]:
#서울교통공사 지하철혼잡도정보
#https://www.data.go.kr/data/15071311/fileData.do
#서울교통공사 1-8호선 30분 단위 평균 혼잡도로 30분간 지나는 열차들의
#평균 혼잡도(정원대비 승차인원으로, 승차인과 좌석수가 일치할 경우를 혼잡도 34%로 산정) 입니다.(단위: %).
#서울교통공사 혼잡도 데이터는 요일구분(평일, 토요일, 일요일), 호선, 역번호, 역명, 상하선구분, 30분단위 별 혼잡도 데이터로 구성되어 있습니다.
#(1년 단위 업데이트 자료 입니다.)
'''
def subwayConfuse():
    url2023 = 'https://api.odcloud.kr/api/15071311/v1/uddi:e477f1d9-2c3a-4dc8-b147-a55584583fa2'
    key = apikey.PBDATA_DECODING_KEY
    params = {'serviceKey':key,'page':1,'perPage':10}
    response = requests.get(url2023,params=params)
    return response
'''

#서울시 지하철 호선별 역별 시간대별 승하차 인원 정보
#https://data.seoul.go.kr/dataList/OA-12252/S/1/datasetView.do
'''
def subwayBDhour(startIdx,endIdx,useMon,line): #useMon = YYYYMM, line = X호선
    key = apikey.SEOULDATA_SUBWAY_KEY
    url = 'http://openapi.seoul.go.kr:8088/' + key + '/xml/CardSubwayTime/' + str(startIdx) + '/' + str(endIdx)
    url += '/' + str(useMon) + '/' + line
    response = requests.get(url)
    return response
'''

"\ndef subwayBDhour(startIdx,endIdx,useMon,line): #useMon = YYYYMM, line = X호선\n    key = apikey.SEOULDATA_SUBWAY_KEY\n    url = 'http://openapi.seoul.go.kr:8088/' + key + '/xml/CardSubwayTime/' + str(startIdx) + '/' + str(endIdx)\n    url += '/' + str(useMon) + '/' + line\n    response = requests.get(url)\n    return response\n"

#### 2-1.날짜 데이터

In [11]:
# 공휴일 확인
# https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15012690
def holidayCheck(year):
    key = apikey.PBDATA_DECODING_KEY
    url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getRestDeInfo'
    params ={'serviceKey':key,'solYear':year,'numOfRows':100} #year:0000, month:00
    response = requests.get(url, params=params)
    resDict = xmltodict.parse(response.content)
    temp_df = pd.DataFrame(resDict['response']['body']['items']['item'])
    temp_df = temp_df.drop(['dateKind','isHoliday','seq'],axis=1)
    return temp_df

def changeDate(x):
    s = x[:4] + '-' + x[4:6] + '-' + x[6:]
    return s

#### 2-2.날씨 데이터

In [19]:
# 일별 기온 데이터
def getTemperature(year):
    if year == 2023:
        t_df = pd.read_csv('subway/서울일별기온/서울일별기온_'+str(year)+'.csv',encoding='EUC-KR')
    else:
        t_df = pd.read_csv('subway/서울일별기온/서울일별기온_'+str(year)+'.csv',encoding='utf-8')
        t_df.columns = ['날짜','지점','평균기온(℃)','최저기온(℃)','최고기온(℃)']
    t_df = t_df.rename(columns={'날짜':'일시','최저기온(℃)':'최저기온','최고기온(℃)':'최고기온'})
    t_df = t_df.drop(['지점','평균기온(℃)'],axis=1)
    return t_df

#### 3. 특별 행사 여부, 인구 통계 및 인프라(업무지구, 학교 등)

In [20]:
def getLocations(address): # 주소 -> 위도, 경도 변환
    geolocator = Nominatim(user_agent="location translator")
    location = geolocator.geocode(address)
    if location:
        latitude = location.latitude
        longitude = location.longitude
        return latitude, longitude
    else:
        return None

In [21]:
# 서울 역별 주위 대학교 여부
# https://namu.wiki/w/%EB%85%B8%EC%84%A0%EB%B3%84%20%EC%A3%BC%EC%9C%84%20%EB%8C%80%ED%95%99%EA%B5%90
def universityStation():
    university = pd.read_csv('subway/대학교위치.csv',encoding='utf-8')

    def getStation(x):
        return x.split()[-1][:-1]

    university['역 이름'] = university['역 이름'].apply(getStation)
    university.drop(['대학','호선'],axis=1,inplace=True)
    vc = university.value_counts()
    university = vc.reset_index()
    university.columns = ['역명', '주변대학수']
    return university

In [22]:
# 서울 행사 정보
# https://data.seoul.go.kr/dataList/OA-15486/S/1/datasetView.do
def seoulFestival(startIdx,endIdx,year):
    key = apikey.SEOULDATA_NOMAL_KEY
    url = 'http://openapi.seoul.go.kr:8088/{}/xml/culturalEventInfo/{}/{}'.format(key,startIdx,endIdx)
    response = requests.get(url)
    return response

In [23]:
key = apikey.SEOULDATA_NOMAL_KEY
startIdx = 1
endIdx = 5
year = 2023
url = 'http://openapi.seoul.go.kr:8088/{}/xml/culturalEventInfo/{}/{}'.format(key,startIdx,endIdx)
res= requests.get(url)
resDict = xmltodict.parse(res.content)

#### 종합 함수

In [24]:
def makeData(year):
    df = subwayDf(year)
    # 공휴일 데이터
    holiday_df = holidayCheck(year)
    holiday_df = holiday_df.rename(columns={'locdate':'일시'})
    holiday_df['일시'] = holiday_df['일시'].apply(changeDate)
    holiday_df['공휴일여부'] = 1
    holiday_df = holiday_df.drop(['dateName'],axis=1)
    df_merged = df.merge(holiday_df, on='일시', how='left').fillna({'공휴일여부':0})
    df_merged['공휴일여부'] = df_merged['공휴일여부'].astype(int)
    # 일별 강수량 데이터
    weather_df = pd.read_csv('subway/서울일별강수량_2020_2023.csv',encoding='EUC-KR')
    weather_df_drop = weather_df.drop(['지점번호','지점명','1시간최다강수량(mm)','1시간최다강수량시각','Unnamed: 6'],axis=1)
    df_merged = df_merged.merge(weather_df_drop, on='일시', how='left').fillna({'강수량(mm)':0})
    # 기온 데이터
    df_merged = df_merged.merge(getTemperature(year), on='일시', how='left')
    # 역별 대학교 수
    df_merged = df_merged.merge(universityStation(), on='역명', how='left').fillna({'주변대학수':0})
    df_merged['주변대학수'] = df_merged['주변대학수'].astype(int)
    return df_merged

In [26]:
df_2023 = makeData(2021)
df_2023.tail(5)

,일시,호선,역번호,역명,주말,요일,통과호선수,미세먼지(pm10),시간,하차인원,공휴일여부,강수량(mm),최저기온,최고기온,주변대학수
1737174,2021-12-31,8,2824,단대오거리,0,4,1,22.0,22,402,0,0.0,-8.8,-3.9,1
1737175,2021-12-31,8,2825,신흥,0,4,1,22.0,22,179,0,0.0,-8.8,-3.9,0
1737176,2021-12-31,8,2826,수진,0,4,1,22.0,22,204,0,0.0,-8.8,-3.9,0
1737177,2021-12-31,8,2827,모란,0,4,2,22.0,22,96,0,0.0,-8.8,-3.9,0
1737178,2021-12-31,8,2828,남위례,0,4,1,22.0,22,138,0,0.0,-8.8,-3.9,0


In [ ]:
#df_test.to_csv('learningdata/testdata_202301.csv',encoding='utf-8',index=False)
#df_2023.to_csv('learningdata/testdata_2023.csv',encoding='utf-8',index=False)